In [24]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Device
device = torch.device("cpu")

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# Simple Feedforward Neural Network
class SNN_BPTT(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(SNN_BPTT, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize
net = SNN_BPTT().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training
epochs = 3
for epoch in range(epochs):
    net.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing
net.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [ ]:
pip uninstall -y torchvision
pip install torchvision==0.18.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu


In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import gzip
import os
import numpy as np
import requests
from torchvision import datasets, transforms


# Device
device = torch.device("cpu")

# Function to download MNIST directly
def download_mnist_file(url, filename):
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        r = requests.get(url)
        with open(filename, "wb") as f:
            f.write(r.content)

base_url = "https://github.com/myleott/mnist_png/raw/master/mnist/"
files = {
    "train_images": "train-images-idx3-ubyte.gz",
    "train_labels": "train-labels-idx1-ubyte.gz",
    "test_images": "t10k-images-idx3-ubyte.gz",
    "test_labels": "t10k-labels-idx1-ubyte.gz"
}

os.makedirs("mnist_data", exist_ok=True)
for key, fname in files.items():
    download_mnist_file(f"http://yann.lecun.com/exdb/mnist/{fname}", os.path.join("mnist_data", fname))

# Load MNIST data
def load_images(path):
    with gzip.open(path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
        data = data.reshape(-1, 28*28).astype(np.float32) / 255.0
        return torch.tensor(data)

def load_labels(path):
    with gzip.open(path, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=8)
        return torch.tensor(data, dtype=torch.long)

x_train = load_images("mnist_data/train-images-idx3-ubyte.gz")
y_train = load_labels("mnist_data/train-labels-idx3-ubyte.gz")
x_test = load_images("mnist_data/t10k-images-idx3-ubyte.gz")
y_test = load_labels("mnist_data/t10k-labels-idx1-ubyte.gz")

train_loader = DataLoader(TensorDataset(x_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test, y_test), batch_size=1000, shuffle=False)

# Simple feedforward network
class SNN_BPTT(nn.Module):
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        super(SNN_BPTT, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = SNN_BPTT().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    net.train()
    running_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Testing
net.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


AttributeError: partially initialized module 'torchvision' has no attribute 'extension' (most likely due to a circular import)

In [28]:
!pip uninstall -y torchvision
!pip install torchvision==0.18.1+cpu --extra-index-url https://download.pytorch.org/whl/cpu


Found existing installation: torchvision 0.17.2+cpu
Uninstalling torchvision-0.17.2+cpu:
  Successfully uninstalled torchvision-0.17.2+cpu
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.4/190.4 MB 28.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2+cpu
    Uninstalling torch-2.2.2+cpu:━━━━━━━━━━━━━━━ 0/2 [torch]
      Successfully uninstalled torch-2.2.2+cpu━━ 0/2 [torch]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchvision] [torchvision]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.2+cpu requires torch==2.2.2, but you have torch 2.3.1+cpu which is incompatible.

[notice] A new release of pip is available: 25.1.1 -> 25.2
[